In [41]:
import numpy as np
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
import json
import os

In [42]:
# Функция считающая процентиль для агрегации
def percentile(n):
    def percentile_(sroki):
        return np.percentile(sroki, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [43]:
def object_to_date(deliv_time, deliv_time_features, name_features):
    deliv_time[name_features] = pd.to_datetime(
        deliv_time_features,
        format="%Y-%m-%dT%H:%M:%S"
    )

In [44]:
headers = {
    'Content-type': 'application/json',
}
data = {
    "НачалоПериода": "19.09.2018 0:00:00",
    "КонецПериода": "19.12.2020 23:59:59",
    "ВидПоставкиТоваров": "От поставщиков"
}

In [45]:
def load_data_from_api(service_url):
    response = requests.post(
        service_url,
        auth=HTTPBasicAuth('Web', 'WebMarket'),
        data=json.dumps(data),
        headers=headers
    )
    return response.json()

In [46]:
list_to_drop = [
    'warehouse_id', 'item_id', 'fabricator_id', 'item_id',
    'item_guid', 'item_charac_id', 'item_charac_guid'
]

In [47]:
# Посмотрим локальную директорию
os.listdir('/Users/2/PycharmProjects/ordersprediction/files')

['delivery_time.csv', 'statistics.json']

In [48]:
# Загрузим датасет
deliv = pd.read_csv('/Users/2/PycharmProjects/ordersprediction/files/delivery_time.csv')

In [49]:
# Проверим 5 рандомных записей
deliv.sample(5)

,date_receipt,warehouse_id,warehouse_guid,item_id,item_guid,item_charac_id,item_charac_guid,date_orders,fabricator_id,fabricator_guid,IsFromSupplier
308,2020-12-19T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,105055,c2a1ef3b-3ffc-11e9-a2d4-000c29675a27,1,cb310891-3ffc-11e9-a2d4-000c29675a27,2020-12-19T00:00:00,16,5464cd6c-b080-11e2-85e4-00155d006702,0
5435,2020-12-22T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,44840,82352cc7-a0bd-11e6-80d2-00155d000e07,1,48dbf4c6-a0bf-11e6-80d2-00155d000e07,2020-12-09T00:00:00,16,5464cd6c-b080-11e2-85e4-00155d006702,1
366,2020-12-19T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,54532,fe0b375e-3c4e-11e7-80d9-00155d000e07,1,fe0b3760-3c4e-11e7-80d9-00155d000e07,2020-12-19T00:00:00,22,1baec3fb-d646-11e2-b394-f46d04736e4a,0
5678,2020-12-22T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,105059,13258675-3ffd-11e9-a2d4-000c29675a27,1,1c460fab-3ffd-11e9-a2d4-000c29675a27,2020-12-13T00:00:00,16,5464cd6c-b080-11e2-85e4-00155d006702,1
3431,2020-12-19T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,42714,53c7dbe5-69f1-11e6-80d2-00155d000e07,2,315fe7a3-e466-11e6-80d6-00155d000e02,2020-12-02T00:00:00,22,1baec3fb-d646-11e2-b394-f46d04736e4a,1


In [50]:
# Дропним ненужные фичи и переменуем нужные
deliv['OFFER_ID'] = deliv[['item_guid', 'item_charac_guid']].apply(lambda x: '#'.join(x), axis=1)
deliv.drop(list_to_drop, axis=1, inplace=True)
deliv.rename(
    columns={
        'warehouse_guid': 'PlacementToId',
        'fabricator_guid': 'PlacementFromId'
    }, inplace=True
)

In [51]:
# Посмотрим число уникальных IsFromSupplier
deliv.IsFromSupplier.unique()

array([0, 1], dtype=int64)

In [52]:
# Выберем только IsFromSupplier == 1
deliv = deliv.loc[deliv['IsFromSupplier'] == 1]

In [53]:
# Посмотрим число уникальных IsFromSupplier
deliv.IsFromSupplier.unique()

array([1], dtype=int64)

In [54]:
# Посмотрим на результат
deliv.sample(10)

,date_receipt,PlacementToId,date_orders,PlacementFromId,IsFromSupplier,OFFER_ID
4246,2020-12-20T00:00:00,a2f89531-75de-11e8-a2c3-000c29dcdf3d,2020-12-17T00:00:00,1a46cb6b-c740-11e6-80d4-00155d000e07,1,1d178c1e-b155-11ea-ab38-000c29a01421#2a0c4004-...
4925,2020-12-21T00:00:00,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-23T00:00:00,c511cfeb-b583-11e7-ab58-000c2945f18b,1,4e6927f3-05f0-11e9-a2c7-00505607eae9#4e6927f5-...
5557,2020-12-22T00:00:00,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-26T00:00:00,5464cd6c-b080-11e2-85e4-00155d006702,1,ca1a7d4c-3ffb-11e9-a2d4-000c29675a27#ca1a7d50-...
5833,2020-12-22T00:00:00,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-29T00:00:00,799adf67-cb79-11e6-80d4-00155d000e07,1,e02f3e45-69c4-11ea-a307-00505607eae9#3d6a071e-...
5407,2020-12-22T00:00:00,f6548095-c106-11e0-8baa-00262d126e8b,2020-12-05T00:00:00,5464cd6c-b080-11e2-85e4-00155d006702,1,82352cc4-a0bd-11e6-80d2-00155d000e07#82352cc6-...
3838,2020-12-20T00:00:00,f6548095-c106-11e0-8baa-00262d126e8b,2020-12-10T00:00:00,3146f195-f3f4-11e9-a2dc-000c29675a27,1,a484eac6-fb04-11e9-a2e0-00505607eae9#a484eac8-...
5671,2020-12-22T00:00:00,f6548095-c106-11e0-8baa-00262d126e8b,2020-12-10T00:00:00,5464cd6c-b080-11e2-85e4-00155d006702,1,13258675-3ffd-11e9-a2d4-000c29675a27#1c460fab-...
5074,2020-12-21T00:00:00,03f70e82-3e4e-11e8-a2bb-000c29dcdf3d,2020-12-01T00:00:00,1b38fc80-5d45-11ea-a307-00505607eae9,1,c7a1bdc3-9124-11ea-ab37-000c29a01421#c7a1bdc9-...
4663,2020-12-21T00:00:00,4bc4d898-ec0e-11e9-a2dc-000c29675a27,2020-11-24T00:00:00,5464cd6c-b080-11e2-85e4-00155d006702,1,62bd4980-3ffd-11e9-a2d4-000c29675a27#68bf1a67-...
5337,2020-12-22T00:00:00,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-16T00:00:00,5464cd6c-b080-11e2-85e4-00155d006702,1,a9cccf97-a0b7-11e6-80d2-00155d000e07#a9cccf99-...


In [58]:
deliv.PlacementFromId.unique()

array(['1baec3fb-d646-11e2-b394-f46d04736e4a',
       '4038bae0-e7bc-11e6-80d6-00155d000e02',
       '4bc0ce74-bb0d-11e3-9031-f46d04736e4a',
       '6903d81b-4454-11e7-8b3e-902b34d492d7',
       '3904ba5a-3bb0-11e7-80d9-00155d000e07',
       '4c809680-62ab-11e6-80d1-00155d000e07',
       '14caa9a8-e4cd-11e9-a2db-000c29675a27',
       '343563b8-3843-11ea-a2fc-00505607eae9',
       '3588e768-874a-11e3-a732-f46d04736e4a',
       '7e1e555b-557c-11e7-ab48-000c2945f18b',
       '1a46cb6b-c740-11e6-80d4-00155d000e07',
       '14c48fc0-5450-11e3-9599-f46d04736e4a',
       'a86462dd-0d90-11e4-95a7-f46d04736e4a',
       'b27d9c73-464d-11ea-a305-00505607eae9',
       '70509cad-bd19-11ea-ab38-000c29a01421',
       'a951b06e-f42d-11e6-80d6-00155d000e02',
       '085f3915-84c3-11e2-be7c-f46d04736e4a',
       'df630ac8-c708-11e8-a2c1-00505607eae9',
       '00bae129-1faf-11e9-a2c8-00505607eae9',
       'ce2d4ffe-2242-11e4-8146-f46d04736e4a',
       '4ff3bba1-74fa-11e2-b38d-f46d04736e4a',
       '4c377

In [144]:
# Отсортируем записи по дате поступления на склад
deliv = deliv.sort_values(by=['date_receipt'])

In [145]:
# Посмотрим на датасет
deliv.head(10)

,date_receipt,PlacementToId,date_orders,PlacementFromId,IsFromSupplier
88752,2020-09-19T00:00:00,03f70e82-3e4e-11e8-a2bb-000c29dcdf3d,2020-08-13T00:00:00,53898bbe-d2b7-11e8-a2c4-00505607eae9,1
88875,2020-09-19T00:00:00,5f219248-fadd-11df-8a53-00262d126e8b,2020-08-03T00:00:00,48452082-00c4-11e8-ab61-000c2945f18b,1
88876,2020-09-19T00:00:00,5f219248-fadd-11df-8a53-00262d126e8b,2020-07-03T00:00:00,48452082-00c4-11e8-ab61-000c2945f18b,1
88877,2020-09-19T00:00:00,f6548095-c106-11e0-8baa-00262d126e8b,2020-07-11T00:00:00,4038bae0-e7bc-11e6-80d6-00155d000e02,1
88878,2020-09-19T00:00:00,f6548095-c106-11e0-8baa-00262d126e8b,2020-07-06T00:00:00,f0287a2c-d6a6-11e4-891c-f46d04736e4a,1
88879,2020-09-19T00:00:00,f6548095-c106-11e0-8baa-00262d126e8b,2020-07-16T00:00:00,f0287a2c-d6a6-11e4-891c-f46d04736e4a,1
88880,2020-09-19T00:00:00,f6548095-c106-11e0-8baa-00262d126e8b,2020-07-26T00:00:00,f0287a2c-d6a6-11e4-891c-f46d04736e4a,1
88881,2020-09-19T00:00:00,f6548095-c106-11e0-8baa-00262d126e8b,2020-07-20T00:00:00,f0287a2c-d6a6-11e4-891c-f46d04736e4a,1
88882,2020-09-19T00:00:00,f6548095-c106-11e0-8baa-00262d126e8b,2020-08-05T00:00:00,86623a51-217a-11ea-a2f3-00505607eae9,1
88883,2020-09-19T00:00:00,f6548095-c106-11e0-8baa-00262d126e8b,2020-07-23T00:00:00,799adf67-cb79-11e6-80d4-00155d000e07,1


In [146]:
# Из типа объект делаем даты
object_to_date(deliv, deliv.date_receipt, 'date_receipt')
object_to_date(deliv, deliv.date_orders, 'date_orders')

In [147]:
# Создадим предиктор со сроками поставок
deliv['difference'] = deliv.date_receipt - deliv.date_orders

# Посмотрим результат
deliv.head(10)

,date_receipt,PlacementToId,date_orders,PlacementFromId,IsFromSupplier,difference
88752,2020-09-19,03f70e82-3e4e-11e8-a2bb-000c29dcdf3d,2020-08-13,53898bbe-d2b7-11e8-a2c4-00505607eae9,1,37 days
88875,2020-09-19,5f219248-fadd-11df-8a53-00262d126e8b,2020-08-03,48452082-00c4-11e8-ab61-000c2945f18b,1,47 days
88876,2020-09-19,5f219248-fadd-11df-8a53-00262d126e8b,2020-07-03,48452082-00c4-11e8-ab61-000c2945f18b,1,78 days
88877,2020-09-19,f6548095-c106-11e0-8baa-00262d126e8b,2020-07-11,4038bae0-e7bc-11e6-80d6-00155d000e02,1,70 days
88878,2020-09-19,f6548095-c106-11e0-8baa-00262d126e8b,2020-07-06,f0287a2c-d6a6-11e4-891c-f46d04736e4a,1,75 days
88879,2020-09-19,f6548095-c106-11e0-8baa-00262d126e8b,2020-07-16,f0287a2c-d6a6-11e4-891c-f46d04736e4a,1,65 days
88880,2020-09-19,f6548095-c106-11e0-8baa-00262d126e8b,2020-07-26,f0287a2c-d6a6-11e4-891c-f46d04736e4a,1,55 days
88881,2020-09-19,f6548095-c106-11e0-8baa-00262d126e8b,2020-07-20,f0287a2c-d6a6-11e4-891c-f46d04736e4a,1,61 days
88882,2020-09-19,f6548095-c106-11e0-8baa-00262d126e8b,2020-08-05,86623a51-217a-11ea-a2f3-00505607eae9,1,45 days
88883,2020-09-19,f6548095-c106-11e0-8baa-00262d126e8b,2020-07-23,799adf67-cb79-11e6-80d4-00155d000e07,1,58 days


In [148]:
# Выберем только количество 
# дней в сроках поставок
lst = []
for item in deliv.difference:
    lst.append(item.days)
deliv['delivery'] = lst

# Посмотрим информацию по датасету
deliv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71219 entries, 88752 to 159970
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   date_receipt     71219 non-null  datetime64[ns] 
 1   PlacementToId    71219 non-null  object         
 2   date_orders      71219 non-null  datetime64[ns] 
 3   PlacementFromId  71219 non-null  object         
 4   IsFromSupplier   71219 non-null  int64          
 5   difference       71219 non-null  timedelta64[ns]
 6   delivery         71219 non-null  int64          
dtypes: datetime64[ns](2), int64(2), object(2), timedelta64[ns](1)
memory usage: 4.3+ MB


In [149]:
# Результат предыдущих операций
deliv.head()

,date_receipt,PlacementToId,date_orders,PlacementFromId,IsFromSupplier,difference,delivery
88752,2020-09-19,03f70e82-3e4e-11e8-a2bb-000c29dcdf3d,2020-08-13,53898bbe-d2b7-11e8-a2c4-00505607eae9,1,37 days,37
88875,2020-09-19,5f219248-fadd-11df-8a53-00262d126e8b,2020-08-03,48452082-00c4-11e8-ab61-000c2945f18b,1,47 days,47
88876,2020-09-19,5f219248-fadd-11df-8a53-00262d126e8b,2020-07-03,48452082-00c4-11e8-ab61-000c2945f18b,1,78 days,78
88877,2020-09-19,f6548095-c106-11e0-8baa-00262d126e8b,2020-07-11,4038bae0-e7bc-11e6-80d6-00155d000e02,1,70 days,70
88878,2020-09-19,f6548095-c106-11e0-8baa-00262d126e8b,2020-07-06,f0287a2c-d6a6-11e4-891c-f46d04736e4a,1,75 days,75


In [151]:
# # Сгруппируем датасет по производителям и складам, агрегирую сроки поставки процентилем
deliv_min = deliv.groupby(['PlacementFromId', 'PlacementToId', 'IsFromSupplier']).agg(
    {'delivery': percentile(50)}).reset_index()
deliv_max = deliv.groupby(['PlacementFromId', 'PlacementToId', 'IsFromSupplier']).agg(
    {'delivery': percentile(90)}).reset_index()

In [152]:
deliv_max['DaysTo'] = deliv_max.delivery
deliv_min['DaysFrom'] = deliv_min.delivery
deliv_max.drop('delivery', axis=1, inplace=True)
deliv_min.drop('delivery', axis=1, inplace=True)

In [153]:
statistics = pd.merge(
    deliv_max, deliv_min,
    on=['PlacementFromId', 'PlacementToId', 'IsFromSupplier'],
    how='left'
)
statistics

,PlacementFromId,PlacementToId,IsFromSupplier,DaysTo,DaysFrom
0,00000000-0000-0000-0000-000000000000,03f70e82-3e4e-11e8-a2bb-000c29dcdf3d,1,63.0,63.0
1,00000000-0000-0000-0000-000000000000,5f219248-fadd-11df-8a53-00262d126e8b,1,50.0,50.0
2,00bae129-1faf-11e9-a2c8-00505607eae9,03f70e82-3e4e-11e8-a2bb-000c29dcdf3d,1,67.3,37.0
3,00bae129-1faf-11e9-a2c8-00505607eae9,385b9ff4-cccf-11ea-ab39-000c29a0142b,1,53.8,51.0
4,00bae129-1faf-11e9-a2c8-00505607eae9,4bc4d898-ec0e-11e9-a2dc-000c29675a27,1,59.8,39.0
...,...,...,...,...,...
1511,fed611d8-cb93-11e4-b853-f46d04736e4a,03f70e82-3e4e-11e8-a2bb-000c29dcdf3d,1,62.3,54.0
1512,fed611d8-cb93-11e4-b853-f46d04736e4a,5f219248-fadd-11df-8a53-00262d126e8b,1,63.0,48.0
1513,fed611d8-cb93-11e4-b853-f46d04736e4a,a2f89531-75de-11e8-a2c3-000c29dcdf3d,1,67.2,56.0
1514,fed611d8-cb93-11e4-b853-f46d04736e4a,f6548095-c106-11e0-8baa-00262d126e8b,1,91.0,65.0


In [154]:
statistics.to_json('statistics.json', orient="split")
statistics.to_excel('statistics.xlsx', index=False)  

### To be continued...

In [155]:
deliv

,date_receipt,PlacementToId,date_orders,PlacementFromId,IsFromSupplier,difference,delivery
88752,2020-09-19,03f70e82-3e4e-11e8-a2bb-000c29dcdf3d,2020-08-13,53898bbe-d2b7-11e8-a2c4-00505607eae9,1,37 days,37
88875,2020-09-19,5f219248-fadd-11df-8a53-00262d126e8b,2020-08-03,48452082-00c4-11e8-ab61-000c2945f18b,1,47 days,47
88876,2020-09-19,5f219248-fadd-11df-8a53-00262d126e8b,2020-07-03,48452082-00c4-11e8-ab61-000c2945f18b,1,78 days,78
88877,2020-09-19,f6548095-c106-11e0-8baa-00262d126e8b,2020-07-11,4038bae0-e7bc-11e6-80d6-00155d000e02,1,70 days,70
88878,2020-09-19,f6548095-c106-11e0-8baa-00262d126e8b,2020-07-06,f0287a2c-d6a6-11e4-891c-f46d04736e4a,1,75 days,75
...,...,...,...,...,...,...,...
159471,2020-12-17,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-18,d86359ec-e37f-11ea-ab3c-000c29a0142b,1,29 days,29
159472,2020-12-17,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-29,d86359ec-e37f-11ea-ab3c-000c29a0142b,1,18 days,18
159473,2020-12-17,f6548095-c106-11e0-8baa-00262d126e8b,2020-12-10,d86359ec-e37f-11ea-ab3c-000c29a0142b,1,7 days,7
159464,2020-12-17,4bc4d898-ec0e-11e9-a2dc-000c29675a27,2020-10-15,9850e147-7c17-11e7-ab50-000c2945f18b,1,63 days,63


In [157]:
deliv.to_csv('delivery_time.csv', index=False)  